In [2]:
import re
from os.path import join
import numpy as np
import pandas as pd
import xarray as xr
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
# location_dirs = glob("/home/patel_zeel/OpenAQ/PurpleAir/data/*")
csv_files = glob("/home/patel_zeel/OpenAQ/PurpleAir/data/**/*.csv.gz", recursive=True)

In [ ]:
# combine all csv files into one csv with ubuntu command
# cat *.csv.gz > combined.csv.gz

In [2]:
def process_file(location_dir):
    csv_files = glob(join(location_dir, "**/*.csv.gz"), recursive=True)
    # print(f"Processing {len(csv_files)} files in {location_dir}")
    combo_df = pd.concat([pd.read_csv(file) for file in csv_files])
    combo_df["lat_lon"] = combo_df["lat"].astype(str) + "_" + combo_df["lon"].astype(str)
    assert len(combo_df.location_id.unique()) == 1, f"location_id assert: {combo_df.location_id.unique()}"
    location_id = combo_df.location_id.iloc[0]
    ds_list = []
    for lat_lon_suffix, lat_lon in enumerate(combo_df["lat_lon"].unique()):
        latlon_id = int(f"{location_id}{lat_lon_suffix}")
        df = combo_df[combo_df["lat_lon"] == lat_lon]
        timezone = re.findall(r"[-+]\d{2}:\d{2}", df["datetime"].iloc[0])[0]
        new_df = df.pivot_table(index=["datetime"], columns="parameter", values="value", aggfunc="mean")
        # new_df.index = pd.to_datetime(new_df.index, utc=True)
        # new_df.index = pd.to_datetime(new_df.index.strftime("%Y-%m-%d %H:%M:%S"))
        ds = new_df.to_xarray().expand_dims("latlon_id")

        # assert len(df.location.unique()) == 1, f"location assert: {df.location.unique()}"
        assert len(df.lat.unique()) == 1, df[["lat", "lon", "location_id", "location"]].drop_duplicates()
        assert len(df.lon.unique()) == 1, np.unique(df[["lat", "lon"]])
        ds["latlon_id"] = [latlon_id]
        ds.coords["location_id"] = ("latlon_id", [df.iloc[0].location_id])
        ds.coords["timezone"] = ("latlon_id", [timezone])
        ds.coords["location"] = ("latlon_id", [df.iloc[0].location])
        ds.coords["lat"] = ("latlon_id", [df.iloc[0].lat])
        ds.coords["lon"] = ("latlon_id", [df.iloc[0].lon])
        ds_list.append(ds)
    # ds = xr.concat(ds_list, dim="latlon_id")
    # convert all data vars to float 16 without a for loop
    # ds = ds.astype(np.float16)
    
    return ds_list
    
np.random.seed(0)
np.random.shuffle(location_dirs)
n_processes = 48
ds_list = Parallel(n_jobs=n_processes)(delayed(process_file)(location_dir) for location_dir in tqdm(location_dirs))

  0%|          | 0/22498 [00:00<?, ?it/s]

/home/patel_zeel/miniconda3/envs/torch_gpu_py311/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [3]:
ds = xr.open_dataset("/home/patel_zeel/OpenAQ/PurpleAir/purpleair_california.nc")
ds

<xarray.Dataset> Size: 4GB
Dimensions:      (datetime: 44350, location_id: 11618)
Coordinates:
  * datetime     (datetime) datetime64[ns] 355kB 2018-03-25T18:00:00 ... 2023...
  * location_id  (location_id) int64 93kB 61935 61938 61942 ... 1637277 1658374
    lat          (location_id) float64 93kB ...
    lon          (location_id) float64 93kB ...
Data variables:
    value        (datetime, location_id) float64 4GB ...

## Appendix

In [5]:
ds = xr.open_zarr("/tmp/zasdjsadelhi/cpcb_camx_ds.zarr.zip")
ds

<xarray.Dataset> Size: 111MB
Dimensions:      (Timestamp: 8760, station: 63)
Coordinates:
  * Timestamp    (Timestamp) datetime64[ns] 70kB 2023-01-01T00:30:00 ... 2023...
    address      (station) <U187 47kB dask.array<chunksize=(63,), meta=np.ndarray>
    city         (station) <U18 5kB dask.array<chunksize=(63,), meta=np.ndarray>
    latitude     (station) float64 504B dask.array<chunksize=(63,), meta=np.ndarray>
    longitude    (station) float64 504B dask.array<chunksize=(63,), meta=np.ndarray>
    state        (station) <U17 4kB dask.array<chunksize=(63,), meta=np.ndarray>
  * station      (station) <U64 16kB 'Alipur, Delhi - DPCC' ... 'Wazirpur, De...
Data variables: (12/26)
    AT           (Timestamp, station) float64 4MB dask.array<chunksize=(2190, 32), meta=np.ndarray>
    BP           (Timestamp, station) float64 4MB dask.array<chunksize=(2190, 32), meta=np.ndarray>
    Benzene      (Timestamp, station) float64 4MB dask.array<chunksize=(2190, 32), meta=np.ndarray>
    CO           (Timestamp, station) float64 4MB dask.array<chunksize=(2190, 32), meta=np.ndarray>
    Eth-Benzene  (Timestamp, station) float64 4MB dask.array<chunksize=(2190, 32), meta=np.ndarray>
    MP-Xylene    (Timestamp, station) float64 4MB dask.array<chunksize=(2190, 32), meta=np.ndarray>
    ...           ...
    TOT-RF       (Timestamp, station) float64 4MB dask.array<chunksize=(2190, 32), meta=np.ndarray>
    Toluene      (Timestamp, station) float64 4MB dask.array<chunksize=(2190, 32), meta=np.ndarray>
    VWS          (Timestamp, station) float64 4MB dask.array<chunksize=(2190, 32), meta=np.ndarray>
    WD           (Timestamp, station) float64 4MB dask.array<chunksize=(2190, 32), meta=np.ndarray>
    WS           (Timestamp, station) float64 4MB dask.array<chunksize=(2190, 32), meta=np.ndarray>
    Xylene       (Timestamp, station) float64 4MB dask.array<chunksize=(2190, 32), meta=np.ndarray>

In [12]:
df = ds.sel(Timestamp="2023-12").isel(station=0).to_dataframe().reset_index()
df["diff"] = df["PM2.5"] - df["P25"]
df.corr(numeric_only=True)['diff']

AT            -0.006902
BP             0.068563
Benzene        0.255110
CO             0.391459
Eth-Benzene         NaN
MP-Xylene           NaN
NH3            0.426099
NO             0.403536
NO2            0.323284
NOx            0.436226
O Xylene            NaN
Ozone         -0.150440
P10           -0.149375
P25           -0.142300
PM10           0.772530
PM2.5          0.866415
RF                  NaN
RH             0.231162
SO2            0.074136
SR             0.087294
TOT-RF              NaN
Toluene        0.182347
VWS                 NaN
WD            -0.151958
WS                  NaN
Xylene         0.321350
latitude            NaN
longitude           NaN
diff           1.000000
Name: diff, dtype: float64